In [1]:
import sympy as sp

def sym_basis_k_symbolic(k, d=3):
    basis = []
    
    # Factorials and sqrt become symbolic
    factorial = sp.factorial
    sqrt = sp.sqrt
    
    def generate(remain, parts):
        if len(parts) == d - 1:
            parts = parts + [remain]
            a, b, c = parts
            
            norm = sqrt(factorial(a) * factorial(b) * factorial(c) / factorial(k))
            basis.append((tuple(parts), sp.simplify(norm)))
            return
        
        for i in range(remain, -1, -1):
            generate(remain - i, parts + [i])

    generate(k, [])
    return basis

# Test
for vec, norm in sym_basis_k_symbolic(4):
    print(vec, norm)


(4, 0, 0) 1
(3, 1, 0) 1/2
(3, 0, 1) 1/2
(2, 2, 0) sqrt(6)/6
(2, 1, 1) sqrt(3)/6
(2, 0, 2) sqrt(6)/6
(1, 3, 0) 1/2
(1, 2, 1) sqrt(3)/6
(1, 1, 2) sqrt(3)/6
(1, 0, 3) 1/2
(0, 4, 0) 1
(0, 3, 1) 1/2
(0, 2, 2) sqrt(6)/6
(0, 1, 3) 1/2
(0, 0, 4) 1


In [2]:
import sympy as sp
from collections import defaultdict
from functools import lru_cache

def pi_symmetric_multinomial_opt(M, basis, numeric=False):
    """
    Compute the symmetric multinomial projection of M on the given basis.

    Args:
        M (sp.Matrix): d x d symbolic matrix.
        basis (list): List of (vector, norm) tuples from sym_basis_k_symbolic.
        numeric (bool): If True, evaluates numerically to speed up large k.

    Returns:
        sp.Matrix: Projected matrix.
    """
    d = M.rows
    n = len(basis)
    piM = sp.zeros(n, n)
    factorial = sp.factorial

    # Precompute factorials up to max k
    max_k = max(sum(vec) for vec, _ in basis)
    factorials = [factorial(i) for i in range(max_k + 1)]

    # Group basis by total degree k
    basis_by_k = defaultdict(list)
    for idx, (vec, norm) in enumerate(basis):
        k = sum(vec)
        basis_by_k[k].append((idx, vec, norm))

    # Precompute M powers
    max_power = max_k
    powers = {}
    for u in range(d):
        for v in range(d):
            powers[(u, v)] = [1]  # M[u,v]**0
            val = 1
            for t in range(1, max_power + 1):
                val *= M[u, v]
                powers[(u, v)].append(val)

    # Function to enumerate contingency matrices
    def enum_contingency(rows, cols):
        @lru_cache(maxsize=None)
        def _recurse(row_idx, cols_remaining):
            cols_remaining = list(cols_remaining)
            if row_idx == len(rows):
                if all(c == 0 for c in cols_remaining):
                    return [()]
                return []
            results = []
            target = rows[row_idx]

            def compose(pos, left, current):
                if pos == len(cols_remaining) - 1:
                    val = left
                    if val <= cols_remaining[pos]:
                        yield tuple(current + [val])
                    return
                maxv = min(left, cols_remaining[pos])
                for v in range(maxv, -1, -1):
                    yield from compose(pos + 1, left - v, current + [v])

            for row_choice in compose(0, target, []):
                new_cols = tuple(cols_remaining[j] - row_choice[j] for j in range(len(cols_remaining)))
                for tail in _recurse(row_idx + 1, new_cols):
                    results.append(tuple(row_choice) + tail)
            return results

        return _recurse(0, tuple(cols))

    # Compute projection
    for k_val, group in basis_by_k.items():
        k_fact = factorials[k_val]
        for i_idx, vec_i, norm_i in group:
            for j_idx, vec_j, norm_j in group:
                val = 0
                for mat_flat in enum_contingency(tuple(vec_i), tuple(vec_j)):
                    term = 1
                    denom = 1
                    for u in range(d):
                        for v in range(d):
                            t = mat_flat[u * d + v]
                            if t:
                                term *= powers[(u, v)][t]
                                denom *= factorials[t]
                    val += k_fact / denom * term
                piM[i_idx, j_idx] = norm_i * norm_j * val
                if numeric:
                    piM[i_idx, j_idx] = sp.N(piM[i_idx, j_idx])

    return piM



In [5]:
# --- Test ---
d = 3
k_val = 6
M = sp.Matrix([[sp.symbols(f'M{i}{j}') for j in range(d)] for i in range(d)])
basis = sym_basis_k_symbolic(k_val, d)
piM = pi_symmetric_multinomial_opt(M, basis)
display(piM)

Matrix([
[                         M00**6,                                                                              sqrt(6)*M00**5*M01,                                                                              sqrt(6)*M00**5*M02,                                                                                                                                                          sqrt(15)*M00**4*M01**2,                                                                                                                                                                                                                                                      sqrt(30)*M00**4*M01*M02,                                                                                                                                                          sqrt(15)*M00**4*M02**2,                                                                                                                                            

In [3]:
import sympy as sp
import random

# assume these functions are already defined from previous code:
# - sym_basis_k_symbolic
# - pi_symmetric_multinomial

def random_int_matrix(d, lo=-3, hi=3):
    """Generate a random integer dxd SymPy matrix."""
    return sp.Matrix([[random.randint(lo, hi) for _ in range(d)] for _ in range(d)])


def test_representation_property(d=3, k_val=4, trials=3):
    print(f"Testing π(AB) = π(A) π(B) for S^{k_val}(ℂ^{d}) ...\n")

    # fixed basis for this k and d
    basis = sym_basis_k_symbolic(k_val, d)

    for t in range(trials):
        print(f"--- Trial {t+1} ---")

        # Random integer matrices A, B
        A = random_int_matrix(d)
        B = random_int_matrix(d)
        AB = A * B

        print("A =", A)
        print("B =", B)

        # Compute symmetric power matrices
        piA = pi_symmetric_multinomial_opt(A, basis)
        piB = pi_symmetric_multinomial_opt(B, basis)
        piAB = pi_symmetric_multinomial_opt(AB, basis)

        # Compare
        diff = sp.simplify(piAB - (piA * piB))

        if diff == sp.zeros(*diff.shape):
            print("✅ PASS:  pi(AB) == pi(A)*pi(B)")
        else:
            print("❌ FAIL!")
            print("Difference:")
            display(diff)
            return False  # early stop

    print("\nAll tests passed ✅")
    return True


# Run the property test
if __name__ == "__main__":
    random.seed(0)
    test_representation_property(d=3, k_val=10, trials=3)

Testing π(AB) = π(A) π(B) for S^10(ℂ^3) ...

--- Trial 1 ---
A = Matrix([[3, 0, 3], [0, -3, -1], [1, 0, 0]])
B = Matrix([[3, 3, -1], [0, -1, 1], [-2, 1, -2]])
✅ PASS:  pi(AB) == pi(A)*pi(B)
--- Trial 2 ---
A = Matrix([[-1, -2, 3], [-3, 1, 3], [-1, 1, 2]])
B = Matrix([[3, 1, -2], [-1, -3, 2], [-3, 3, 2]])
✅ PASS:  pi(AB) == pi(A)*pi(B)
--- Trial 3 ---
A = Matrix([[-1, 0, 1], [-3, -1, 0], [-1, 1, 2]])
B = Matrix([[-2, 1, 0], [0, 3, 1], [-1, -3, 3]])
✅ PASS:  pi(AB) == pi(A)*pi(B)

All tests passed ✅
